# Wasserstein GAN - MNIST

In [1]:
%matplotlib inline
import importlib
import utils2; importlib.reload(utils2)
from utils2 import *

from tqdm import tqdm

/opt/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import torch_utils; importlib.reload(torch_utils)
from torch_utils import *
import torch.nn
import torch.nn.parallel
import torch.nn.functional as F

from torch import FloatTensor as FT

In [3]:
#Batchsize, image size and size of noise vector
bs,sz,nz = 64,28,100

In [4]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
X_train = np.expand_dims(X_train, -1); X_train.shape

(60000, 28, 28, 1)

In [6]:
X_train=X_train.reshape((60000, 1, 28, 28))

In [7]:
X_traint = torch.from_numpy(X_train)
y_traint = torch.from_numpy(y_train)

In [8]:
X_traint = X_traint.float()

In [9]:
datas = torch.utils.data.TensorDataset(X_traint, y_traint)

In [10]:
dataloader = torch.utils.data.DataLoader(datas, bs, True)

In [11]:
n = len(dataloader); n

938

In [12]:
y_train.shape

(60000,)

### Generator

In [27]:
 class GAN_G(nn.Module):
        #inF: number of input features/channels
        #outF: number of output channels
        #kS: kernel size
        #strd: strides
        #p: padding
        #uOut: Output height width of upsampling 
        #uOutF: Features output of upsampling
        def up_block(self, uOutF, outF, uOut, kS, strd, p, bn=True):
            upblock = nn.Sequential(
                nn.Upsample((uOutF, uOut, uOut)),
                nn.Conv2d(uOutF, outF, kS, strd, p, bias=False),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(outF)
            )
            return upblock
        def __init__(self, sz, nz):
            super(GAN_G, self).__init__()
            self.input_height = 28
            self.input_width = 28
            self.input_dim = 100
            self.output_dim = 1
           

            self.fc = nn.Sequential(
                nn.Linear(self.input_dim, 1024),
                nn.BatchNorm1d(1024),
                nn.ReLU(),
                nn.Linear(1024, 128 * (self.input_height // 4) * (self.input_width // 4)),
                nn.BatchNorm1d(128 * (self.input_height // 4) * (self.input_width // 4)),
                nn.ReLU(),
            )
            self.deconv = nn.Sequential(
                nn.ConvTranspose2d(128, 64, 4, 2, 1),
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.ConvTranspose2d(64, self.output_dim, 4, 2, 1),
                nn.Sigmoid(),
            )            
            '''self.initial = nn.Sequential(
            nn.Linear(nz, 512*7*7),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm2d(512*7*7),
            )
            
            self.upblock1 = nn.Sequential(
                nn.Conv2d(512, 64, 3, 1, 1, bias=False),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(64)
            )
            self.upblock2 = nn.Sequential(
                nn.Conv2d(64, 32, 3, 1, 1, bias=False),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(32)
            )
            
            self.final = nn.Sequential(
                nn.Conv2d(32, 1, 1, bias=False),
                nn.Tanh(),
            )  '''
        def forward(self, input):
            '''x = self.initial(input)
            x = x.view(-1, 512, 7, 7)
            x = F.upsample(x, scale_factor=2)
            x = self.upblock1(x)
            x = F.upsample(x, scale_factor=2)
            x = self.upblock2(x)
            x = self.final(x)
            return x'''
            
            x = self.fc(input)
            x = x.view(-1, 128, (self.input_height // 4), (self.input_width // 4))
            x = self.deconv(x)

            return x      

### Discriminator

In [28]:
class GAN_D(nn.Module):

    def __init__(self, nc=1):
        super(GAN_D, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 256, 5, 2, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(256, 512, 5, 2, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )
        
        self.dense = nn.Sequential(
            nn.Linear(512*4*4, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
        )
        
        
    def forward(self, input):
        x = self.conv1(input)
        x = self.conv2(x)
        x = x.view(-1, 512*4*4)
        x = self.dense(x)
        return x

### Weight functions

In [29]:
def weights_init(m):
    if isinstance(m, (nn.Conv2d)): 
        m.weight.data.normal_(0.0, 0.02)
    elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

### Training

In [30]:
netG = GAN_G(sz, nz).cuda()
netG.apply(weights_init);

In [31]:
netD = GAN_D(1).cuda()
netD.apply(weights_init);

In [32]:
fake = netG(create_noise(real.size()[0]))

NameError: name 'real' is not defined

In [35]:
def Var(*parms): return Variable(FT(*parms).cuda())

In [36]:
def create_noise(b): 
    return Variable(FT(b, nz).cuda().normal_(0, 1))

In [37]:
# Input placeholder
input = Var(bs, 100)
# Fixed noise used just for visualizing images when done
fixed_noise = create_noise(bs)
# The numbers 0 and -1
one = np.array([1]*bs).astype('float32')
one = torch.from_numpy(one).cuda()
one = one.view(64,1)
mone = one * -1

In [38]:
optimizerD = optim.RMSprop(netD.parameters(), lr = 1e-5)
optimizerG = optim.RMSprop(netG.parameters(), lr = 1e-5)

In [39]:
G_params = list(netG.parameters())
D_params = list(netD.parameters())

In [40]:
def step_D(v, init_grad):
    err = netD(v)
    err = torch.mean(err)
    err.backward(init_grad)
    return err

In [41]:
def make_trainable(net, val): 
    for p in net.parameters(): p.requires_grad = val

### Train loop

In [42]:
def train(niter, first=True):
    gen_iterations = 100
    for epoch in tqdm(range(niter)):
        data_iter =  iter(dataloader)
        i = 0
        while i < n:
            make_trainable(netD, True)
            d_iters = (100 if first and (gen_iterations < 25) or gen_iterations % 500 == 0 
                       else 5)
            j = 0.01
            while j < d_iters and i < n:
                j+=1; i+=100
                real = Variable(next(data_iter)[0].cuda())
                input.data.resize_(real.size())
                netD.zero_grad()
                errD_real = netD(real)
                
                fake = netG(create_noise(real.size()[0]))
                input.data.resize_(real.size()).copy_(fake.data)
                errD_fake = netD(fake)
                dloss = -errD_real.mean() + errD_fake.mean()
                dloss.backward()
                optimizerD.step()
                for p in netD.parameters(): p.data.clamp_(-0.01, 0.01)

            
            make_trainable(netD, False)
            netG.zero_grad()
            errD_fake = netD(netG(create_noise(bs)))
            gloss = -errD_fake.mean()
            gloss.backward()
            optimizerG.step()
            
            if gen_iterations % 300 == 0:
                print('Iter-{}; D_loss: {}; G_loss: {}'
                .format(gen_iterations, dloss.cpu().data.numpy(), gloss.cpu().data.numpy()))
            gen_iterations += 1
                

### Test

In [ ]:
real = Variable(next(iter(dataloader))[0])

In [ ]:
%time train(1500, True)

  7%|▋         | 101/1500 [00:28<06:36,  3.53it/s]

Iter-300; D_loss: [-0.7304528]; G_loss: [0.27716875]


 17%|█▋        | 251/1500 [01:12<05:59,  3.47it/s]

Iter-600; D_loss: [-1.3493488]; G_loss: [0.58862776]


 27%|██▋       | 401/1500 [01:56<05:19,  3.44it/s]

...

 47%|████▋     | 701/1500 [03:27<03:56,  3.38it/s]

Iter-1500; D_loss: [-1.4016883]; G_loss: [0.61653864]


 57%|█████▋    | 851/1500 [04:11<03:12,  3.38it/s]

Iter-1800; D_loss: [-1.4118255]; G_loss: [0.61541766]


 59%|█████▊    | 880/1500 [04:20<03:03,  3.38it/s]

In [ ]:
fake = netG(fixed_noise).data.cpu()

In [ ]:
import torchvision
from torchvision.utils import make_grid

In [ ]:
%matplotlib inline
def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')


In [ ]:
show(torchvision.utils.make_grid(fake))

In [ ]:
one = X_train[9].reshape(28,28)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(one, interpolation='nearest')
plt.show()

In [ ]:
plt.hist(one.ravel(),256,[0,256]); plt.show()